In [ ]:
# xmlrpc_server.ipynb

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler

import xmlrpc.client

import datetime
import pandas as pd
import pickle

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8008), 
                            requestHandler=RequestHandler)

server.register_introspection_functions()

stats_server = xmlrpc.client.ServerProxy("http://localhost:8018")

# Добавление в лог через сервер
def add_log(log_line):
    try:
        stats_server.add_log(log_line)
    except:
        print("Сервер статистики не доступен")

# Тест
def ping():
    add_log("ping")
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    add_log("now")
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    add_log("show_type")
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    add_log("test_sum")
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    add_log("test_pov")
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname, name, patronym, birth):
    add_log("black_list_check")
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] + frame['Name'] + frame['Patronym'] + frame['Birth']  == sname + name + patronym + birth)
    if (exist == True):
        return sname, name, patronym, birth + ": "+ "bad_boy"
    else:
        return sname, name, patronym, birth + ": "+ "good_boy"

server.register_function(black_list_check, 'black_list_check')

# Бинарная передача данных
def send_back_binary(bin_data):
    add_log("send_back_binary")
    data = bin_data.data
    return xmlrpc.client.Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

# Инверсия цвета
# На вход изображение размерности (M, N, 3) и (M, N, 1)
def send_back_inversion(bin_data):
    add_log("send_back_inversion")
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1] 
    r = len(img_arr.shape) 
    if r == 3:
        for i in range(height):
            for j in range(width):
                img_arr[i][j][0] = 255 - img_arr[i][j][0]
                img_arr[i][j][1] = 255 - img_arr[i][j][1]
                img_arr[i][j][2] = 255 - img_arr[i][j][2]
    else:
        for i in range(height):
            for j in range(width):
                img_arr[i][j] = 255 - img_arr[i][j]
                
    pimg = pickle.dumps(img_arr)        
    return xmlrpc.client.Binary(pimg)

server.register_function(send_back_inversion, 'color_inversion')



# Бинаризация изображения
# На вход изображение размерности (M, N, 3) и (M, N, 1)
def send_back_bin(bin_data, n):
    add_log("send_back_bin")
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1] 
    r = len(img_arr.shape) 
     
    if r == 3:
        for i in range(height):
            for j in range(width):
                t = 0.3333 * img_arr[i][j][0] + 0.3333 * img_arr[i][j][1] + 0.3333 * img_arr[i][j][2]
                img_arr[i][j][0] = t
                img_arr[i][j][1] = t
                img_arr[i][j][2] = t
        
    if r == 2: 
        for i in range(height):
            for j in range(width):
                if img_arr[i][j] < n:
                    img_arr[i][j] = 0
                else:
                    img_arr[i][j] = 225   
        
      
    else:
        for i in range(height):
            for j in range(width):
                if img_arr[i][j][0] > n:
                    img_arr[i][j][0] = 255 
                elif img_arr[i][j][0] < n:
                    img_arr[i][j][0] = 0 
                if img_arr[i][j][1] > n:
                    img_arr[i][j][1] = 255 
                elif img_arr[i][j][1] < n:
                    img_arr[i][j][1] = 0   
                if img_arr[i][j][2] > n:
                    img_arr[i][j][2] = 255 
                elif img_arr[i][j][2] < n:
                    img_arr[i][j][2] = 0
        
      
    pimg = pickle.dumps(img_arr)        
    return xmlrpc.client.Binary(pimg)

server.register_function(send_back_bin, 'bin_i')



# Разворот изображения относительно вертикали
def send_back_vturn(bin_data):
    add_log("send_back_vturn")
    img_arr = pickle.loads(bin_data.data)
    
    r = len(img_arr.shape) 
    if r == 3:                         #(M, N, 3)
        img_arr = img_arr[:,::-1,:]
    else:                              #(M, N, 1)
        img_arr = img_arr[:,::-1]
                
    pimg = pickle.dumps(img_arr)        
    return xmlrpc.client.Binary(pimg)

server.register_function(send_back_vturn, 'vertical_turn')


print ("Listening on port 8008...")
server.serve_forever()



Listening on port 8008...


127.0.0.1 - - [22/Nov/2023 18:10:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 18:10:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 18:10:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 18:10:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 18:10:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 18:10:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 18:10:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 18:10:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 18:10:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 18:10:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 18:10:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 18:10:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 18:10:44] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 18:11:03] "POST /RPC2 HTTP/1.1" 200 -
